# ResNet50 with ImageNet weights






In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Importing the libraries

In [0]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input
from keras.models import load_model, model_from_json

Using TensorFlow backend.


## Part 1 - Data Preprocessing

### Generating images for the Training set

In [0]:
train_datagen = ImageDataGenerator(shear_range = 10,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   preprocessing_function = preprocess_input)

### Creating the Training set

In [0]:
training_generator = train_datagen.flow_from_directory('/content/drive/My Drive/SI_Projekt/dataset/training_set/',
                                                 target_size = (224, 224),
                                                 shuffle = False,
                                                 class_mode = 'categorical')

Found 150 images belonging to 10 classes.


### Generating images for the Validation set

In [0]:
validation_datagen = ImageDataGenerator(shear_range = 10,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   preprocessing_function = preprocess_input)

### Creating the Validation set

In [0]:
validation_generator = train_datagen.flow_from_directory('/content/drive/My Drive/SI_Projekt/dataset/test_set/',
                                                 target_size = (224, 224),
                                                 shuffle = False,
                                                 class_mode = 'categorical')

Found 70 images belonging to 10 classes.


## Part 2 - Building the ResNet50 CNN

In [0]:
from keras.applications import ResNet50
from keras import Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

### Initialising

In [0]:
conv_base = ResNet50(
    include_top=False,
    weights='imagenet')

for layer in conv_base.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
x = conv_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x) 
predictions = Dense(10, activation='softmax')(x)
model = Model(conv_base.input, predictions)

### Compiling

In [0]:
optimizer = Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## Part 3 - Training


### Training on the Training set and Validating



In [0]:
from keras.callbacks import EarlyStopping

In [0]:
early_callback = EarlyStopping(monitor='loss', patience=3)
model.fit(x=training_generator,
                              steps_per_epoch=75,  
                              epochs=5,
                              validation_data=validation_generator,
                              validation_steps=35,
                              callbacks = [early_callback]
                             )

Epoch 1/5
75/75 [==============================] - 623s 8s/step - loss: 1.1058 - accuracy: 0.6978 - val_loss: 1.2971 - val_accuracy: 0.6103
Epoch 2/5
75/75 [==============================] - 624s 8s/step - loss: 0.1072 - accuracy: 0.9991 - val_loss: 1.1131 - val_accuracy: 0.6361
Epoch 3/5
75/75 [==============================] - 620s 8s/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 1.0158 - val_accuracy: 0.6250
Epoch 4/5
75/75 [==============================] - 625s 8s/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 1.5935 - val_accuracy: 0.6343
Epoch 5/5
75/75 [==============================] - 617s 8s/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 1.2937 - val_accuracy: 0.6374


# Save and load the model

In [0]:
!mkdir models
!mkdir models/keras

Save the model (Architecture and weights in HDF5)

In [0]:
model.save('models/keras/malarzy.h5')

Load the model

In [0]:
model = load_model('models/keras/malarzy.h5')

# Make predictions on sample test images

In [0]:
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image

In [0]:
test_images = [f for f in os.listdir('/content/drive/My Drive/SI_Projekt/dataset/test_images') if os.path.isfile(os.path.join('/content/drive/My Drive/SI_Projekt/dataset/test_images', f))]

img_list = [Image.open('/content/drive/My Drive/SI_Projekt/dataset/test_images/' + img_name) for img_name in test_images]

In [0]:
validation_batch = np.stack([preprocess_input(np.array(img.resize((224,224))))
                             for img in img_list])

In [0]:
pred_probs = model.predict(validation_batch)

In [0]:
for i, img in enumerate(img_list):
    title = "{:.0f}% Chagall, {:.0f}% Dali, {:.0f}% Gauguin, {:.0f}% Hokusai, {:.0f}% Malevich, {:.0f}% Picasso, {:.0f}% Raphael, {:.0f}% Repin, {:.0f}% Seurat, {:.0f}% Warhol"
    print(title.format(100*pred_probs[i,0], 100*pred_probs[i,1], 100*pred_probs[i,2],
                                         100*pred_probs[i,3], 100*pred_probs[i,4], 100*pred_probs[i,5],
                                         100*pred_probs[i,6], 100*pred_probs[i,7], 100*pred_probs[i,8],
                                         100*pred_probs[i,9]))
    display(img)

Output hidden; open in https://colab.research.google.com to view.